In [1]:
#Training Part
import os
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,ReduceLROnPlateau,EarlyStopping,TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from unet import build_unet
from metrics import dice_loss, dice_coef

(None, 256, 256, 64) (None, 128, 128, 128) (None, 64, 64, 256) (None, 32, 32, 512)
(None, 128, 128, 64) (None, 64, 64, 128) (None, 32, 32, 256) (None, 16, 16, 512)
<KerasTensor shape=(None, 256, 256, 1), dtype=float32, sparse=False, name=keras_tensor_67>


Model: "UNET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]  

 Total params: 31,055,297 (118.47 MB)

 Trainable params: 31,043,521 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nikhilroxtomar/brain-tumor-segmentation")

print("Path to dataset files:", path)

100%|██████████| 312M/312M [00:01<00:00, 218MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/nikhilroxtomar/brain-tumor-segmentation/versions/1


In [3]:
import os
os.getcwd()

'/content'

In [2]:
## Global parameters
W=256
H=256

In [3]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


In [4]:
def load_dataset(path,split=0.2):
    images=sorted(glob(os.path.join(path,"images","*.png")))
    masks=sorted(glob(os.path.join(path,"masks","*.png")))
    split_size=int(len(images)*split)
    train_x,valid_x=train_test_split(images,test_size=split_size,random_state=42)
    train_y,valid_y=train_test_split(masks,test_size=split_size,random_state=42)
    train_x,test_x=train_test_split(train_x,test_size=split_size,random_state=42)
    train_y,test_y=train_test_split(train_y,test_size=split_size,random_state=42)
    return (train_x,train_y),(valid_x,valid_y),(test_x,test_y)
dataset_path="/root/.cache/kagglehub/datasets/nikhilroxtomar/brain-tumor-segmentation/versions/1"
(train_x,train_y),(valid_x,valid_y),(test_x,test_y)=load_dataset(dataset_path)

In [5]:
def read_image(path):
    path=path.decode()
    x=cv2.imread(path,cv2.IMREAD_COLOR)
    x=cv2.resize(x,(W,H))
    x=x/255.0 # Black Remains  0 as 0/255 is 0 (Normalized)
    x=x.astype(np.float32)
    return x


In [6]:
def read_mask(path):
    path=path.decode()
    x=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    x=cv2.resize(x,(W,H))
    x=x/255.0 # White Becomes 1 as 255/255 is 1 (Normalized)
    x=x.astype(np.float32)
    x=np.expand_dims(x,axis=-1)
    return x


In [7]:
def tf_parse(x,y):
    def parse(x,y):
        x=read_image(x)
        y=read_mask(y)
        return x,y
    x,y=tf.numpy_function(parse,[x,y],[tf.float32,tf.float32])
    x.set_shape([H,W,3])
    y.set_shape([H,W,1])
    return x,y


In [8]:
def tf_dataset(x,y,batch=2):
    dataset=tf.data.Dataset.from_tensor_slices((x,y))
    dataset=dataset.map(tf_parse)
    dataset=dataset.batch(batch)
    dataset=dataset.prefetch(10)
    return dataset


In [9]:
## Seeding for same randomness always
np.random.seed(42)
tf.random.set_seed(42)
## Directory for storing files
create_dir("files")

In [10]:
## Hyperparameters
batch_size=16
lr=1e-4
num_epochs=50
model_path=os.path.join("files","model.h5")
csv_path=os.path.join("files","log.csv")

In [11]:
train_dataset=tf_dataset(train_x,train_y,batch=batch_size)
valid_dataset=tf_dataset(valid_x,valid_y,batch=batch_size)
# for x,y in train_dataset:
#     print(x.shape,y.shape)

In [12]:
model=build_unet((H,W,3))
model.compile(loss=dice_loss,optimizer=Adam(lr),metrics=[dice_coef])
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks
)

(None, 256, 256, 64) (None, 128, 128, 128) (None, 64, 64, 256) (None, 32, 32, 512)
(None, 128, 128, 64) (None, 64, 64, 128) (None, 32, 32, 256) (None, 16, 16, 512)
<KerasTensor shape=(None, 256, 256, 1), dtype=float32, sparse=False, name=keras_tensor_135>
Epoch 1/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 805ms/step - dice_coef: 0.0721 - loss: 0.9279
Epoch 1: val_loss improved from inf to 0.97901, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - dice_coef: 0.0723 - loss: 0.9277 - val_dice_coef: 0.0209 - val_loss: 0.9790 - learning_rate: 1.0000e-04
Epoch 2/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 808ms/step - dice_coef: 0.1724 - loss: 0.8276
Epoch 2: val_loss did not improve from 0.97901
115/115 ━━━━━━━━━━━━━━━━━━━━ 125s 988ms/step - dice_coef: 0.1725 - loss: 0.8275 - val_dice_coef: 0.0188 - val_loss: 0.9811 - learning_rate: 1.0000e-04
Epoch 3/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 806ms/step - dice_coef: 0.2271 - loss: 0.7729
Epoch 3: val_loss improved from 0.97901 to 0.97665, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 119s 1s/step - dice_coef: 0.2272 - loss: 0.7728 - val_dice_coef: 0.0233 - val_loss: 0.9767 - learning_rate: 1.0000e-04
Epoch 4/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 806ms/step - dice_coef: 0.2927 - loss: 0.7073
Epoch 4: val_loss improved from 0.97665 to 0.86959, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 131s 940ms/step - dice_coef: 0.2928 - loss: 0.7072 - val_dice_coef: 0.1299 - val_loss: 0.8696 - learning_rate: 1.0000e-04
Epoch 5/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 806ms/step - dice_coef: 0.3660 - loss: 0.6340
Epoch 5: val_loss improved from 0.86959 to 0.85931, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 119s 1s/step - dice_coef: 0.3662 - loss: 0.6338 - val_dice_coef: 0.1407 - val_loss: 0.8593 - learning_rate: 1.0000e-04
Epoch 6/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 807ms/step - dice_coef: 0.4376 - loss: 0.5624
Epoch 6: val_loss improved from 0.85931 to 0.72736, saving model to files/model.h5


...

115/115 ━━━━━━━━━━━━━━━━━━━━ 111s 963ms/step - dice_coef: 0.9268 - loss: 0.0732 - val_dice_coef: 0.8120 - val_loss: 0.1853 - learning_rate: 1.0000e-05
Epoch 37/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 808ms/step - dice_coef: 0.9292 - loss: 0.0708
Epoch 37: val_loss improved from 0.18534 to 0.18485, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - dice_coef: 0.9292 - loss: 0.0708 - val_dice_coef: 0.8124 - val_loss: 0.1848 - learning_rate: 1.0000e-05
Epoch 38/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 808ms/step - dice_coef: 0.9311 - loss: 0.0689
Epoch 38: val_loss improved from 0.18485 to 0.18471, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 131s 958ms/step - dice_coef: 0.9311 - loss: 0.0689 - val_dice_coef: 0.8125 - val_loss: 0.1847 - learning_rate: 1.0000e-05
Epoch 39/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 808ms/step - dice_coef: 0.9326 - loss: 0.0674
Epoch 39: val_loss did not improve from 0.18471
115/115 ━━━━━━━━━━━━━━━━━━━━ 136s 907ms/step - dice_coef: 0.9326 - loss: 0.0674 - val_dice_coef: 0.8124 - val_loss: 0.1848 - learning_rate: 1.0000e-05
Epoch 40/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 807ms/step - dice_coef: 0.9339 - loss: 0.0661
Epoch 40: val_loss did not improve from 0.18471
115/115 ━━━━━━━━━━━━━━━━━━━━ 151s 987ms/step - dice_coef: 0.9339 - loss: 0.0661 - val_dice_coef: 0.8120 - val_loss: 0.1851 - learning_rate: 1.0000e-05
Epoch 41/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 807ms/step - dice_coef: 0.9351 - loss: 0.0649
Epoch 41: val_loss improved from 0.18471 to 0.18450, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 111s 962ms/step - dice_coef: 0.9351 - loss: 0.0649 - val_dice_coef: 0.8126 - val_loss: 0.1845 - learning_rate: 1.0000e-05
Epoch 42/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 807ms/step - dice_coef: 0.9362 - loss: 0.0638
Epoch 42: val_loss did not improve from 0.18450
115/115 ━━━━━━━━━━━━━━━━━━━━ 113s 987ms/step - dice_coef: 0.9362 - loss: 0.0638 - val_dice_coef: 0.8123 - val_loss: 0.1847 - learning_rate: 1.0000e-05
Epoch 43/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 807ms/step - dice_coef: 0.9373 - loss: 0.0627
Epoch 43: val_loss did not improve from 0.18450
115/115 ━━━━━━━━━━━━━━━━━━━━ 133s 906ms/step - dice_coef: 0.9373 - loss: 0.0627 - val_dice_coef: 0.8124 - val_loss: 0.1847 - learning_rate: 1.0000e-05
Epoch 44/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 808ms/step - dice_coef: 0.9383 - loss: 0.0617
Epoch 44: val_loss improved from 0.18450 to 0.18448, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 149s 970ms/step - dice_coef: 0.9383 - loss: 0.0617 - val_dice_coef: 0.8126 - val_loss: 0.1845 - learning_rate: 1.0000e-05
Epoch 45/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 807ms/step - dice_coef: 0.9393 - loss: 0.0607
Epoch 45: val_loss did not improve from 0.18448
115/115 ━━━━━━━━━━━━━━━━━━━━ 113s 987ms/step - dice_coef: 0.9393 - loss: 0.0607 - val_dice_coef: 0.8124 - val_loss: 0.1846 - learning_rate: 1.0000e-05
Epoch 46/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 808ms/step - dice_coef: 0.9404 - loss: 0.0596
Epoch 46: val_loss did not improve from 0.18448

Epoch 46: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
115/115 ━━━━━━━━━━━━━━━━━━━━ 104s 907ms/step - dice_coef: 0.9404 - loss: 0.0596 - val_dice_coef: 0.8123 - val_loss: 0.1847 - learning_rate: 1.0000e-05
Epoch 47/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 807ms/step - dice_coef: 0.9411 - loss: 0.0589
Epoch 47: val_loss did not improve from 0.18448
115/115 ━━━━━━━━━━━━━━━━━━━━ 151s 987ms/step - dice_coe